In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [2]:
parse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df_1 = pd.read_csv('takehome_user_engagement.csv',parse_dates = ['time_stamp'], date_parser=parse ,encoding = 'ISO-8859-1')
df_1['time_stamp_1'] = df_1['time_stamp']
df_1['user_id_1'] = df_1['user_id']
df_1['time_stamp_2'] = df_1['time_stamp']
df_1['user_id_2'] = df_1['user_id']
df_1['time_stamp_1'] = df_1['time_stamp'].shift(1)
df_1['user_id_1'] = df_1['user_id'].shift(1)
df_1['time_stamp_2'] = df_1['time_stamp'].shift(2)
df_1['user_id_2'] = df_1['user_id'].shift(2)
df_1['user_id'] = df_1['user_id'].astype(float)
del df_1['visited']
df_1['Time Difference'] = df_1['time_stamp'] - df_1['time_stamp_2']
df_1.loc[((df_1['user_id_2'] == df_1['user_id']) & ((timedelta(days=-1) < df_1['Time Difference']) & (df_1['Time Difference'] < timedelta(days=8)))), 'Adopted User'] = True
adopt = df_1.groupby('user_id')[['Adopted User']].count()

In [3]:
df = pd.read_csv('takehome_users.csv',parse_dates = ['creation_time'], date_parser=parse, encoding = 'ISO-8859-1')
adopt.loc[adopt['Adopted User'] > 0, 'Adopted User'] = 1
adopt.loc[adopt['Adopted User'] < 1, 'Adopted User'] = 0
adopt['object_id'] = adopt.index
adopt.reset_index(drop=True, inplace=True)
adopt['object_id'].astype(int)
adopt
df = pd.DataFrame.merge(df,adopt,on='object_id')
df.dropna
df = pd.get_dummies(df, columns=['creation_source'])
df['creation_month'] = df['creation_time'].dt.month
df['creation_day'] = df['creation_time'].dt.day
df

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,Adopted User,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,creation_month,creation_day
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,1.398139e+09,1,0,11,10803.0,0,1,0,0,0,0,4,22
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,0,0,1,316.0,1,0,1,0,0,0,11,15
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,1.363735e+09,0,0,94,1525.0,0,0,1,0,0,0,3,19
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,1.369210e+09,0,0,1,5151.0,0,1,0,0,0,0,5,21
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,1.358850e+09,0,0,193,5240.0,0,1,0,0,0,0,1,17
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,1.387424e+09,0,0,197,11241.0,0,1,0,0,0,0,12,17
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,1.356010e+09,0,1,37,NaN,0,0,0,0,1,0,12,16
7,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,1.401833e+09,1,1,318,4143.0,1,0,1,0,0,0,1,16
8,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,1.388117e+09,0,0,69,NaN,0,0,0,0,1,0,12,26
9,13,2014-03-30 16:19:38,Fry Alexander,AlexanderDFry@cuvox.de,1.396196e+09,0,0,254,11204.0,0,0,1,0,0,0,3,30


In [8]:
clf = LogisticRegression(max_iter=500000, C=10.0)
X = df[['creation_month','creation_day', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'creation_source_GUEST_INVITE','creation_source_ORG_INVITE','creation_source_PERSONAL_PROJECTS','creation_source_SIGNUP','creation_source_SIGNUP_GOOGLE_AUTH']]
y = df[['Adopted User']].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
clf.predict_proba(X_test)
rfr = RandomForestClassifier()
rfr.fit(X_train, y_train)
print(rfr.score(X_test, y_test))
clf.coef_

0.81148469966
0.744994333207


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


array([[ 0.03985244, -0.00670671,  0.07341115, -0.03963341, -0.00068741,
        -0.35280572, -0.1169475 , -0.497729  , -0.36731574]])